In [61]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql as sql 
from sklearn.preprocessing import StandardScaler , MinMaxScaler , LabelEncoder
import joblib

In [62]:
cnt=sql.connect(
    user='root',
    password='123456',
    host='localhost',
    database='xyz_holidays'
)
cur=cnt.cursor()
cur.execute("select * from travel;")
data=cur.fetchall()
cols=[i[0] for i in cur.description]
TR=pd.DataFrame(data=data,columns=cols)

In [63]:
#TR.columns

In [64]:
#TR.head(1)

In [65]:
#int('')

In [66]:
TR['Age']=['0' if i=='' else i for i in TR['Age']]

In [70]:
TR['Age']=TR['Age'].astype(int)

In [71]:
#for i in TR.columns:
 #   if TR[i].dtypes != 'object':
  #      sns.histplot(TR[i])
   #     plt.show()

In [72]:
#TR["Gender"]=TR["Gender"].replace("Fe Male","Female")

In [73]:
#for i in TR.columns:
 #   if TR[i].dtypes=="object":
  #      sns.countplot(x=TR[i])
   #     plt.show()

In [74]:
#for i in TR.columns:
 #   if TR[i].dtypes!="object":
  #      sns.boxplot(TR[i])
   #     plt.show()

In [75]:
#for i in TR.columns:
 #   if TR[i].dtypes!="object":
  #      sns.kdeplot(TR[i])
   #     plt.show()

In [76]:
#TR.isnull().sum()

1 replace [Mean | Median | Mode]

2 B fill | F fill 

3 domain specific knowledge

4 interpolation



In [77]:
for i in TR.columns:
    if TR[i].dtypes=='object':
        TR[i].fillna(TR[i].mode()[0],inplace=True)
    else:
        TR[i].fillna(TR[i].mean(),inplace=True)

C:\Users\Ducat\AppData\Local\Temp\ipykernel_6304\2642884581.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  TR[i].fillna(TR[i].mean(),inplace=True)
C:\Users\Ducat\AppData\Local\Temp\ipykernel_6304\2642884581.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when 

In [78]:
#TR.isnull().sum()

# Outliers 

IQR METHOD 

In [79]:
TR1= TR.copy()

In [80]:
for i in TR.columns:
    if TR[i].dtypes != 'object' and i != 'ProdTaken':
        Q1 = TR[i].quantile(0.25)
        Q3 = TR[i].quantile(0.75)
        IQR = Q3-Q1
        UF = Q3+(1.5*IQR)
        LF = Q1-(1.5*IQR)
        TR=TR[TR[i].between(LF,UF)]

In [81]:
TR.shape[0]/TR1.shape[0]

0.8428030303030303

# Standardization

In [82]:
TR2 = TR1.copy()

In [83]:
STD= StandardScaler()

In [84]:
for i in TR1.columns:
    if TR1[i].dtypes!='object':
        TR1[i+"std"]= STD.fit_transform(TR1[[i]])
        TR1= TR1[TR1[i+'std'].between(-3,3)]
        TR1.drop(i+'std',axis=1,inplace=True)

In [85]:
TR1.shape[0] / TR2.shape[0]

0.9732481060606061

In [86]:
LE =  LabelEncoder()

In [87]:
np.unique(LE.fit_transform(TR1["TypeofContact"]))

array([0, 1])

In [88]:
LE.classes_

array(['Company Invited', 'Self Enquiry'], dtype=object)

In [89]:
Labels = []
Encodings= []
for i in TR1.columns:
    if TR1[i].dtypes=='object':
        TR1[i]= LE.fit_transform(TR1[i])
        Labels.append(LE.classes_)
        Encodings.append(np.unique(LE.fit_transform(TR1[i])))

In [90]:
TR1

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41,1,3,6,2,1,3,3,1,3,2,1,1,2,1,0,2,20993
1,200001,0,49,0,1,14,2,2,3,4,1,4,0,2,0,3,1,2,2,20130
2,200002,1,37,1,1,8,0,2,3,4,0,3,2,7,1,3,0,0,1,17090
3,200003,0,33,0,1,9,2,1,2,3,0,3,0,2,1,5,1,1,1,17909
5,200005,0,32,0,1,8,2,2,3,3,0,3,2,1,0,5,1,1,1,18068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4219,204883,1,49,1,3,9,3,2,3,5,1,4,3,2,1,1,1,1,2,26576
4220,204884,1,28,0,1,31,2,2,4,5,0,3,2,3,1,3,1,2,1,21212
4221,204885,1,52,1,3,17,2,1,4,4,3,4,1,7,0,1,1,3,3,31820
4222,204886,1,19,1,3,16,3,2,3,4,0,3,2,3,0,5,0,2,1,20289


In [91]:
Labels=list(zip(TR.select_dtypes('object').columns,Labels))

In [92]:
list(zip(TR.select_dtypes('object').columns,Encodings))

[('TypeofContact', array([0, 1])),
 ('Occupation', array([0, 1, 2, 3])),
 ('Gender', array([0, 1, 2])),
 ('ProductPitched', array([0, 1, 2, 3, 4])),
 ('MaritalStatus', array([0, 1, 2, 3])),
 ('Designation', array([0, 1, 2, 3, 4]))]

In [93]:
Labels

[('TypeofContact', array(['Company Invited', 'Self Enquiry'], dtype=object)),
 ('Occupation',
  array(['Free Lancer', 'Large Business', 'Salaried', 'Small Business'],
        dtype=object)),
 ('Gender', array(['Fe Male', 'Female', 'Male'], dtype=object)),
 ('ProductPitched',
  array(['Basic', 'Deluxe', 'King', 'Standard', 'Super Deluxe'],
        dtype=object)),
 ('MaritalStatus',
  array(['Divorced', 'Married', 'Single', 'Unmarried'], dtype=object)),
 ('Designation',
  array(['AVP', 'Executive', 'Manager', 'Senior Manager', 'VP'],
        dtype=object))]

# Model 

In [31]:
TR1.drop('CustomerID',axis=1,inplace=True)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
x= TR1.drop('ProdTaken',axis=1)
y= TR1.ProdTaken

In [34]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25)

In [35]:
from sklearn.linear_model import LogisticRegression

In [43]:
LR=LogisticRegression(max_iter=500)

In [44]:
LR.fit(xtrain,ytrain)

C:\Users\Ducat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [60]:
LR.predict([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])[0]

C:\Users\Ducat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


np.int64(1)

In [49]:
LR.predict(xtest)

array([0, 0, 0, ..., 0, 0, 0], shape=(1052,))

In [51]:
LR.score(xtest,ytest)

0.8384030418250951

In [52]:
LR.score(xtrain,ytrain)

0.8449587824984147

In [55]:
joblib.dump(LR,'model.pkl')

['model.pkl']

In [58]:
d={"kunal":1}

In [59]:
d["kunal"]

1